In [107]:
import numpy as np
import os
import glob
import itertools
import re
from scipy.fftpack import fft

In [108]:
def Data2ComplexBiasCorrected(filepath, dcI=2025, dcQ=2025):
    # Ref: https://github.com/dhruboroy29/MATLAB_Scripts/blob/6ded2d8d434239e3488ee79e02232dd7afff908c/Scripts/Data2IQ.m
    # Read IQ streams from data
    assert os.path.splitext(filepath)[1] == '.data' or os.path.splitext(filepath)[1] == '.bbs'
    comp = np.fromfile(filepath, dtype=np.uint16).astype(np.int16)
    I = comp[::2]
    Q = comp[1::2]
    assert len(I) == len(Q)

    # Sanity check of I and Q samples (>4096, or abruptly different from prev. sample?)
    for i in range(1,len(I)-1):
        if I[i]>4096 or abs(int(I[i])-int(I[i-1]))>2000 and abs(int(I[i])-int(I[i+1]))>1500:
            I[i] = I[i-1]
        if Q[i]>4096 or abs(int(Q[i])-int(Q[i-1]))>2000 and abs(int(Q[i])-int(Q[i+1]))>1500:
            Q[i] = Q[i-1]
    
    return (I-dcI) + 1j*(Q-dcQ)

In [109]:
def ComputeSTFT(X, stride=64, nfft=256, winlen=256):
    numsteps = (len(X)-winlen)//stride + 1
    
    # Prepare data for FFT
    Y=np.lib.stride_tricks.as_strided(X, (winlen,numsteps), strides=(X.itemsize, stride*X.itemsize))
    
    return fft(Y, n=nfft, axis=0)

## STFT Test Code

On MATLAB, variables saved as follows:
`save('trial.mat', '-v7.3')`

```
import h5py
f=h5py.File('trial.mat')

X = f['X'].value.view(np.complex)[0]
Z = ComputeSTFT(X)
```

Obtain MATLAB FFT to compare:

```
se = f['se'].value.view(np.complex)[0]
```

In [110]:
def extract_windows(file, winlen, stride=None):
        
    if stride is None:
        stride = winlen
    
    # Read complex data
    X = Data2ComplexBiasCorrected(file)
    
    numsteps = (len(X)-winlen)//stride + 1
    Y = np.lib.stride_tricks.as_strided(X, (numsteps,winlen), strides=(stride*X.itemsize, X.itemsize))
    
    # Validate windows
    j = 0
    for i in np.arange(len(Y)):
        assert (X[j*stride:j*stride+winlen] == Y[i]).all()
        j = j+1
        
    return X, Y

## Windows extraction test code

```
Data_cut, Data_win = extract_windows('/mnt/6b93b438-a3d4-40d2-9f3d-d8cdbb850183/Research/PedBike/final_bike_radial_full_cuts/a13_c1_Dec 9 2018_Bike_1p_cut4.data', 512)
```

In [111]:
files=list(itertools.chain(*[glob.glob(os.path.join(r, '*.data'))
                             for r in glob.glob('final_*_radial_full_cuts')]))

# Window length
win_len = 512
win_stride = 128

In [112]:
Data_win_fft = []
label_win_fft = []

for cur_file in files:    
    # Extract labels
    fname=os.path.basename(cur_file)
    flabel=int(re.match('.*_(?P<label>[0-9]+)p_.*', fname).groupdict()['label'])
    
    _, Data_win = extract_windows(cur_file, win_len, stride=win_stride)
    
    for index in np.arange(0, len(Data_win)):
        Data_win_fft.append(ComputeSTFT(Data_win[index]))
        label_win_fft.append(flabel)
    
    # File order of processing, and extracted labels
    #print('{}\t{}'.format(fname, flabel))

In [113]:
Data_win_fft = np.array(Data_win_fft)
label_win_fft = np.array(label_win_fft)

print(Data_win_fft.shape)
print(label_win_fft.shape)

(18642, 256, 5)
(18642,)


In [114]:
np.save('Data_win_fft.npy', Data_win_fft)
np.save('label_win_fft.npy', label_win_fft)